In [1]:
import pandas as pd
data = {

"data":[

                 {"Row Labels": "1/1/2020",  "Plan Invest":2141783263, "Pd Invest": 7187398187, "SAP Payments (Dr)": 0, "SAP Accruals (Cr)": 23232323},

                 #//Other rows of table

             ]

}

#This serialized table should not include target column. Is it "SAP Payments (Dr)"?

 



In [5]:
#Generally we recommend using script for testing:
target_column_name = "SAP Payments (Dr)"  # Is it a column we are trying to predict?

X_test = pd.read_csv('./paid_vs_plan_analysis.csv')

y_test = X_test.pop(target_column_name).values

 
time_column_name = 'Row Labels'
X_test

,Row Labels,Plan Invest,Pd Invest,SAP Accruals (Cr)
0,3/1/20,267266020,172952361,-206936356
1,4/1/20,195255520,103860986,-203834251


In [9]:
import json
import requests

X_query = X_test.copy()

# We have to convert datetime to string, because Timestamps cannot be serialized to JSON.

X_query[time_column_name] = X_query[time_column_name].astype(str)

# The Service object accept the complex dictionary, which is internally converted to JSON string.

# The section 'data' contains the data frame in the form of dictionary.

test_sample = json.dumps({'data': X_query.to_dict(orient='records')})

print(test_sample)
scoring_uri = 'http://ec7350d5-9257-4882-a98d-9602dfb5cff1.westus.azurecontainer.io/score'


headers = {'Content-Type': 'application/json'}
response = requests.post(scoring_uri, test_sample, headers=headers)

# translate from networkese to datascientese
print(response.json())
res_dict = json.loads(response.json())

try:

    
    y_fcst_all = pd.DataFrame(res_dict['index'])

    y_fcst_all[time_column_name] = pd.to_datetime(y_fcst_all[time_column_name], unit = 'ms')

    y_fcst_all['forecast'] = res_dict['forecast']   

except:

    print(res_dict)
    
    print(y_fcst_all)

 

 

{"data": [{"Row Labels": "3/1/20", "Plan Invest": 267266020, "Pd Invest": 172952361, "SAP Accruals (Cr)": -206936356}, {"Row Labels": "4/1/20", "Plan Invest": 195255520, "Pd Invest": 103860986, "SAP Accruals (Cr)": -203834251}]}
{"forecast": [173614521.42621133, 129139915.80414319], "index": [{"Row Labels": 1583020800000, "_automl_dummy_grain_col": "_automl_dummy_grain_col"}, {"Row Labels": 1585699200000, "_automl_dummy_grain_col": "_automl_dummy_grain_col"}]}
